<a href="https://colab.research.google.com/github/boysbytes/colab-chatbot/blob/main/colab_qwen2_5_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [6]:
import os

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create ollama_models folder if it doesn't exist
ollama_models_path = '/content/drive/MyDrive/ollama_models'
if not os.path.exists(ollama_models_path):
    os.makedirs(ollama_models_path)
    print(f"Created directory: {ollama_models_path}")
else:
    print(f"Directory already exists: {ollama_models_path}")

Mounted at /content/drive
Directory already exists: /content/drive/MyDrive/ollama_models


In [7]:
!curl -fsSL https://ollama.com/install.sh | sh

# Install libraries
!pip install ollama
!pip install gradio

import threading
import subprocess
import time

# Start Ollama in a separate process
def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

running = subprocess.run(["pgrep", "-a", "ollama"], capture_output=True, text=True)
print("Ollama Status:", running.stdout if running.stdout else "Ollama is NOT running!")

# Create symlink
!rm -rf ~/.ollama/models
!ln -s /content/drive/MyDrive/ollama_models ~/.ollama/models


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Ollama Status: 1961 ollama serve



### Download the model to Google Drive

In [8]:
import os
model_info_path = '/content/drive/MyDrive/ollama_models/manifests/registry.ollama.ai/library/qwen2.5/0.5b'
if not os.path.exists(model_info_path):
    !ollama pull qwen2.5:0.5b
else:
    print("Model already exists. Skipping download.")

Model already exists. Skipping download.


### Refine your prompt template

In [9]:
# Define prompt template
PROMPT_TEMPLATE = """You are a helpful and friendly mentor.
Your responses should be concise and appropriate for a 15-year-old student.

Conversation History:
{chat_history_formatted}

User's Message: {user_message}

Chatbot's Response:
"""
print("Prompt template loaded successfully.")

Prompt template loaded successfully.


### Define the chatbot function

In [10]:
import ollama
import gradio as gr

# Chat function
def chatbot_response(user_message, chat_history):
    # Ensure chat_history is a list of tuples
    if not isinstance(chat_history, list):
        chat_history = []

    # Format chat history
    chat_history_formatted = "\\n".join([f"{role}: {msg}" for role, msg in chat_history])

    # Format the prompt using the template
    prompt = PROMPT_TEMPLATE.format(chat_history_formatted=chat_history_formatted, user_message=user_message)

    try:
        # Run inference
        response = ollama.chat(model="qwen2.5:0.5b", messages=[{"role": "user", "content": prompt}])
        bot_message = response["message"]["content"]
    except Exception as e:
        bot_message = f"Error: {e}"

    # Update chat history
    chat_history.append(("User", user_message))
    chat_history.append(("Chatbot", bot_message))

    return chat_history, chat_history

### Define the chatbot interface

In [11]:
with gr.Blocks() as chatui:
    gr.Markdown("# 🤖 Qwen2.5 0.5B Chatbot (Google Colab)")

    chat_history = gr.State([])

    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="Your Message", placeholder="Ask me anything!")
    send_btn = gr.Button("Send")

    send_btn.click(chatbot_response, inputs=[msg, chat_history], outputs=[chatbot, chat_history])

# Launch the chatbot UI
chatui.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:284: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://80b710a4cd8c6384ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
